In [1]:
import numpy as np
import utils
import qiskit
import matplotlib.pyplot as plt
import pandas as pd
%load_ext autoreload
%autoreload 2

In [2]:
provider = qiskit.IBMQ.load_account()
simulator = qiskit.Aer.get_backend('qasm_simulator')

## Testing metrics

In [10]:
lima5_result = utils.result_from_csv('measured_data/linear5_lima.csv')

In [13]:
linear_interactions = {(i, i + 1): 1 for i in range(4)}
linear_circ = utils.linear_circuit(5, linear_interactions)
linear_circ.measure_all()
job_sim = qiskit.execute(linear_circ, simulator, shots=8192)
sim_result = job_sim.result().get_counts()

Each function returns its metric and the boolean flag, which is True if result contains all possible observations and False otherwise

In [19]:
utils.kullback_leibler_divergence(sim_result, linear_interactions),\
utils.kullback_leibler_divergence(lima5_result, linear_interactions)

((0.002199073889893586, False), (0.3345491018909429, True))

In [24]:
utils.hellinger_distance(sim_result, linear_interactions),\
utils.hellinger_distance(lima5_result, linear_interactions)

((0.023498918269350376, False), (0.26936296772384316, True))

In [26]:
utils.total_variation_distance(sim_result, linear_interactions),\
utils.total_variation_distance(lima5_result, linear_interactions)

((0.016322934274533046, False), (0.29839518663330333, True))

## Measuring on real IBM data

In [3]:
circuits = []
for n in range(1, 16):
    linear_interactions = {(i, i + 1): 1 for i in range(n - 1)}
    linear_circ = utils.linear_circuit(i, linear_interactions)
    linear_circ.measure_all()
    circuits.append(linear_circ)

In [10]:
backend = provider.get_backend('ibmq_16_melbourne')
job = qiskit.execute(circuits, backend, shots=8192)
qiskit.tools.job_monitor(job)

Job Status: job has successfully run


KeyboardInterrupt: 

In [11]:
results = []
for i in range(1, 16):
    results.append(utils.result_from_csv(f'measured_data/melbourne_linear/{i}.csv'))

In [12]:
measures = []
for n, result in enumerate(results, 1):
    interactions = {(i, i + 1): 1 for i in range(n - 1)}
    probability_denomenator = utils.Z(n, interactions, beta=1)
    
    job_sim = qiskit.execute(circuits[n - 1], simulator, shots=8192)
    sim_result = job_sim.result().get_counts()
    
    ibm_kl, all_obtained = utils.kullback_leibler_divergence(result, interactions, probability_denomenator)
    sim_kl, sim_all_obtained = utils.kullback_leibler_divergence(sim_result, interactions, probability_denomenator)
    
    ibm_l2, _ = utils.hellinger_distance(result, interactions, probability_denomenator)
    sim_l2, _ = utils.hellinger_distance(sim_result, interactions, probability_denomenator)
    
    ibm_tv, _ = utils.total_variation_distance(result, interactions, probability_denomenator)
    sim_tv, _ = utils.total_variation_distance(sim_result, interactions, probability_denomenator)
    
    measures.append([n, ibm_kl, sim_kl, ibm_l2, sim_l2, ibm_tv, sim_tv, all_obtained, sim_all_obtained])

In [13]:
df = pd.DataFrame(measures, columns=['n', 'IBM KL', 'SIM KL', 'IBM L2', 'SIM L2', 'IBM TV', 'SIM TV', 'All obtained', 'Sim all obtained'])
df

,n,IBM KL,SIM KL,IBM L2,SIM L2,IBM TV,SIM TV,All obtained,Sim all obtained
0,1,0.005342,0.000142,0.036573,0.005956,0.051636,0.008423,True,True
1,2,0.081075,0.000020,0.134827,0.002244,0.146788,0.002044,True,True
2,3,0.226552,0.000403,0.212241,0.009943,0.234910,0.008103,True,True
3,4,0.437194,0.000885,0.287081,0.015071,0.282055,0.010875,True,True
4,5,0.619743,0.002543,0.345037,0.024799,0.371669,0.015254,True,False
5,6,0.918285,0.003492,0.408567,0.027007,0.441235,0.017956,True,False
6,7,1.194131,0.007374,0.463867,0.038307,0.511753,0.030331,True,False
7,8,1.300959,0.016422,0.495115,0.051682,0.537188,0.038681,True,False
8,9,1.609797,0.021848,0.542326,0.058442,0.586338,0.042686,False,False
9,10,1.829783,0.041829,0.563799,0.077273,0.613012,0.060508,False,False


## Estemating D-Wave beta

In [2]:
import dimod
from dwave.system import DWaveSampler
from dwave.system import EmbeddingComposite

In [24]:
from scipy.optimize import minimize
from tqdm import tqdm

In [3]:
sampler = DWaveSampler() 
embedding_sampler = EmbeddingComposite(sampler)

In [32]:
opt_betas = []
for n in tqdm(range(1, 16)):
    interactions = {(i, i + 1): 1 for i in range(n - 1)}
    result = utils.result_from_csv(f'measured_data/dwave_linear/{n}.csv')
    
    res_kl = minimize(lambda beta: utils.kullback_leibler_divergence(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_kl, val_kl = res_kl.x[0], res_kl.fun
    
    res_l2 = minimize(lambda beta: utils.hellinger_distance(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_l2, val_l2 = res_l2.x[0], res_l2.fun
    
    res_tv = minimize(lambda beta: utils.total_variation_distance(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_tv, val_tv = res_tv.x[0], res_tv.fun
    
    opt_betas.append([n, beta_kl, val_kl, beta_l2, val_l2, beta_tv, val_tv])

100%|██████████| 15/15 [02:00<00:00,  8.03s/it]


In [31]:
hard_opt_betas = []
for n in tqdm([50, 100, 200, 500, 1000, 1500, 2000]):
    interactions = {(i, i + 1): 1 for i in range(n - 1)}
    result = utils.result_from_csv(f'measured_data/dwave_linear/{n}.csv')
    
    res_kl = minimize(lambda beta: utils.kullback_leibler_divergence(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_kl, val_kl = res_kl.x[0], res_kl.fun
    
    res_l2 = minimize(lambda beta: utils.hellinger_distance(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_l2, val_l2 = res_l2.x[0], res_l2.fun
    
    res_tv = minimize(lambda beta: utils.total_variation_distance(result, interactions, beta=beta[0])[0], 1, bounds=[(0.0, 100.0)])
    beta_tv, val_tv = res_tv.x[0], res_tv.fun
    
    opt_betas.append([n, beta_kl, val_kl, beta_l2, val_l2, beta_tv, val_tv])

 29%|██▊       | 2/7 [03:23<08:29, 101.99s/it]


KeyboardInterrupt: 

In [33]:
df = pd.DataFrame(opt_betas, columns=['n', 'Beta KL', 'Metric KL', 'Beta L2', 'Metric L2', 'Beta TV', 'Metric TV'])
df

,n,Beta KL,Metric KL,Beta L2,Metric L2,Beta TV,Metric TV
0,1,1.000000,0.002406,1.000000,0.024532,1.000000,0.034668
1,2,3.955569,0.011882,3.965918,0.054597,4.137315,0.076904
2,3,3.953390,0.012516,3.807066,0.055560,3.811821,0.078369
3,4,4.155424,0.009992,3.966650,0.049297,4.158517,0.069580
4,5,4.099532,0.000358,4.046365,0.007214,4.158394,0.008789
5,6,3.864661,0.002838,3.764011,0.024249,3.811086,0.033937
6,7,3.519834,0.025958,3.572426,0.078964,3.607371,0.108646
7,8,3.714602,0.039918,3.774113,0.098811,3.810595,0.136721
8,9,3.559589,0.012935,3.653125,0.055121,3.810349,0.073001
9,10,3.239627,0.011399,3.304938,0.051101,3.348613,0.063867
